In [1]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import LambdaLR
import torch.optim as optim
import os
import csv
import random
import copy
import seaborn as sns
import matplotlib.pyplot as plt

seed = 2710
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
with open('data/realworld_mobiact.pkl', 'rb') as f:
    x, y, k = pickle.load(f)
with open('data/realworld_mobiact_fs.pkl', 'rb') as f:
    fs = pickle.load(f)

print(x.shape, y.shape, k.shape, fs.shape)

(22599, 3, 128) (22599,) (22599,) (22599,)


In [3]:
mask_rw = (k < 15)
mask_ma = (k >= 15) & (fs == 0)

x_rw, y_rw, k_rw, fs_rw = x[mask_rw], y[mask_rw], k[mask_rw], fs[mask_rw]
x_ma, y_ma, k_ma, fs_ma = x[mask_ma], y[mask_ma], k[mask_ma], fs[mask_ma]

print(x_rw.shape, y_rw.shape, k_rw.shape, fs_rw.shape)
print(x_ma.shape, y_ma.shape, k_ma.shape, fs_ma.shape)

(11783, 3, 128) (11783,) (11783,) (11783,)
(9596, 3, 128) (9596,) (9596,) (9596,)


In [4]:
x_rw, x_ma, y_rw, y_ma = train_test_split(x_rw, y_rw, test_size=0.2, random_state=seed, stratify=y_rw, shuffle=True)

print(x_rw.shape, y_rw.shape, x_ma.shape, y_ma.shape)

(9426, 3, 128) (9426,) (2357, 3, 128) (2357,)


In [5]:
unique_rw, counts_rw = np.unique(y_rw, return_counts=True)
print("RW:", dict(zip(unique_rw, counts_rw)))

unique_ma, counts_ma = np.unique(y_ma, return_counts=True)
print("MA:", dict(zip(unique_ma, counts_ma)))

RW: {0: 2880, 1: 2586, 2: 1822, 3: 2138}
MA: {0: 720, 1: 647, 2: 455, 3: 535}


In [ ]:
x_train = torch.tensor(x_rw, dtype=torch.float32, device=device)
y_train = torch.tensor(y_rw, dtype=torch.long, device=device)

train_data = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

x_test = torch.tensor(x_ma, dtype=torch.float32, device=device)
y_test = torch.tensor(y_ma, dtype=torch.long, device=device)

In [ ]:
class TSTRClassifier(nn.Module):
    def __init__(self, num_timesteps=128, num_channels=3, num_classes=4):
        super(TSTRClassifier, self).__init__()

        self.conv1 = nn.Conv1d(num_channels, 16, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(16)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(32)
        self.conv3 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(64)
        self.conv4 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn4 = nn.BatchNorm1d(128)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.25)

        self.fc_shared = nn.Linear(num_timesteps * 8, 100)

        self.fc_class = nn.Linear(100, num_classes)

    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        x = self.pool(self.relu(self.bn4(self.conv4(x))))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.dropout(x)
        x = self.relu(self.fc_shared(x))

        # Final output for class prediction
        class_outputs = self.fc_class(x)
        return class_outputs


def test(model, x_test, y_test):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()
    with torch.no_grad():
        x_test, y_test = x_test.to(device), y_test.to(device)
        outputs = model(x_test)
        _, predicted = torch.max(outputs, 1)
        acc = accuracy_score(y_test.cpu().numpy(), predicted.cpu().numpy())
        f1 = f1_score(y_test.cpu().numpy(), predicted.cpu().numpy(), average='weighted')
        print(f'Test Accuracy: {acc:.4f}, F1: {f1:.4f}')
        
        # Compute confusion matrix
        cm = confusion_matrix(y_test.cpu().numpy(), predicted.cpu().numpy())
        print('Confusion Matrix:')
        print(cm)
        
        # Plot confusion matrix
        plt.figure(figsize=(10, 7))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.show()
        

def train(model, train_loader, optimizer, num_epochs=20):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    for epoch in range(num_epochs):
        model.train()
        for i, (x, y) in enumerate(train_loader):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(x)
            loss = nn.CrossEntropyLoss()(outputs, y)
            loss.backward()
            optimizer.step()
        print(f'\nEpoch {epoch}, Loss: {loss.item():.4f}')
    print('\nFinished Training')


for i in range(10):

    model = TSTRClassifier()
        
    lr = 1e-4
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train(model, train_loader, optimizer, num_epochs=20)

    test(model, x_test, y_test)